# 需要的 Python 技巧

在开始讨论具体的梯度性质讨论前，作一些背景了解。这些背景包括 Python、NumPy、`np.einsum`，以及 PySCF 和 Gaussian 的格式化 Checkpoint 文件的使用。这些都是后续讨论中基础的基础。我们先介绍 Python 相关问题。

之所以说这份文档是技巧，是因为在课题的实战中，通常不需要对 Python 的大局有完整的了解——事实上那也是不可能的；且不论 PyPI 社区支持的第三方库有将近 20 万之多，其官方库的内容也非常庞大。在不能了解全局的情况下，我们需要一些投机取巧的方法以最小的编程能力完成重要的任务，不论是否完成得漂亮。这就是这一节文档的目的。

由于这不是一份新手上路的笔记，而是介绍一些经验技巧，因此最好阅读者有基础的 Python 能力，或者可以通过其它语言就可以快速熟悉 Python 的语法．如果读者已经了解这些技巧，那么就权当复习，在后续的笔记中很可能会非常经常地使用这些技巧；一部分技巧未必在教程中提及，这可能是作者的经验教训。如果读者还没有基础的 Python 能力，那么可以通过搜索引擎查阅资料并理解代码，这比阅读书籍会快很多，尽管知识面会不成系统．

读者也可以参考各种 Python 的教程，譬如 [廖雪峰的教程](https://www.liaoxuefeng.com/wiki/1016959663602400)；尽管教程通常都是用心写且高质量的，但很可能你会发现，在实战中遇到问题再请教教程往往是更常见的情况。对于稍有编程经验的初学者，教程更像是文献摘要，允许你在不了解语言的情况下一览语言特性，并了解哪些语言特性是自己完全不了解的、工作中哪些语言特性最有可能使用、以及遇到新的语言特性时不会慌张并能使用大致正确的名词进行 Google。

在这一份文档中，我们会使用 NumPy、Matplotlib 库 (package)。我们也会使用一些其它的 Python 官方库 copy、pickle。我们在下面的代码块 (Jupyter notebook cell) 中将它们引入 (import)。

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import pickle

## 迭代器 (Iterator) 与循环

Python 的循环可以非常灵活。列表 (list)、字典 (dict)、元组 (tuple)、numpy 向量 (ndarray)、范围 (range) 等都是合法的迭代器 (iterator)。

### 范围 (range)

最经典的循环任务是从 1 到 100 相加得到 5050。这样一类任务通常由范围来实现：

In [2]:
summation = 0
for i in range(1, 101):
    summation += i
summation

5050

<div class="alert alert-warning">

**提示**

Python 的范围使用类似于“尾后指针”的迭代方式；即对于 Python 的

    for i in range(a, b):
    
而言，这等同于 C 语言的

    for (int i = a; i < b; ++i)

或者如果 `a`, `b` 分别指代一个 C++ 可迭代容器 `vector<T> vec` 的首指针 `vec.begin()` 与尾指针 `vec.end()`，那么等同于

    for (vector<T>::iterator i = vec.begin(); i < vector.end(); ++i)

由于 `b = vector.end()` 所指代的指针并不在列表 `vec` 中，它是列表最后一个元素之后的指针，因此也称为“尾后指针”。

对于 C++ 编程者而言，Python 的迭代方式可能比较直观；但对于 Fortran 编程者，这需要适应。

</div>

<div class="alert alert-info">

**任务**

范围迭代器不仅可以传入两个参数指定迭代下界与上界，还可以传入一个或三个参数。实践中，一个参数的情况更为常用。

1. 请编写程序，使下述代码输出 5050；填入的代码只能包含数字：

    ```python
    summation = 0
    for i in range("""insert your code here"""):
        summation += i
    print(summation)  # give 5050
    ```

</div>

### 列表 (List)

列表是最常见的数据组合形式。在 Python 中，列表的数据形式未必是相同的；这使得 Python 的列表 (包括元组、字典) 相比于 C++ 等编译语言，使用上非常便利。其使用方式可以是

In [3]:
lst = [1, np, range(1, 10, 3), "Pastafarianism"]
for item in lst:
    print(item, "; type:", type(item))

1 ; type: <class 'int'>
<module 'numpy' from '/mnt/d/Anaconda3WSL/lib/python3.7/site-packages/numpy/__init__.py'> ; type: <class 'module'>
range(1, 10, 3) ; type: <class 'range'>
Pastafarianism ; type: <class 'str'>


列表套列表可以用来构建高维列表。最简单的高维列表是二维列表：

In [4]:
lst = [[], [], []]
for i in range(3):
    for j in range(3):
        lst[i].append(10 * i + j)
lst

[[0, 1, 2], [10, 11, 12], [20, 21, 22]]

生成列表不仅可以使用传统的循环，还可以使用影视循环，避免多行多缩进的代码，使代码更精炼：

In [5]:
lst = [[10 * i + j for j in range(3)] for i in range(3)]
lst

[[0, 1, 2], [10, 11, 12], [20, 21, 22]]

<div class="alert alert-info">

**任务**

1. 请交换上述代码块中的 `for j` 与 `for i`，并观察结果．以后在书写隐式多重循环时，要特别注意指标顺序．

</div>

有时，我们要写高维度的循环；但由于 Python 语法中，缩进数量会随着循环维度的增高而增大，也许会使得代码从排版上难看不少。我们可以通过预先生成循环索引的方式来解决这个问题。以刚才的二维情况来举例。

In [6]:
loop_list = [[i, j]
             for i in range(3)
             for j in range(3)]
print(loop_list)

for i, j in loop_list:
    print(10 * i + j)

[[0, 0], [0, 1], [0, 2], [1, 0], [1, 1], [1, 2], [2, 0], [2, 1], [2, 2]]
0
1
2
10
11
12
20
21
22


其中，`for i, j in loop_list` 的 `i, j` 实际上是元组。我们马上就会提到。 

### 元组 (Tuple)

元组通常是逗号分隔、圆括号或者没有括号组合而成的：

In [7]:
tup = (1, np, range(1, 10, 3), "Pastafarianism")
for item in tup:
    print(item, "; type:", type(item))

1 ; type: <class 'int'>
<module 'numpy' from '/mnt/d/Anaconda3WSL/lib/python3.7/site-packages/numpy/__init__.py'> ; type: <class 'module'>
range(1, 10, 3) ; type: <class 'range'>
Pastafarianism ; type: <class 'str'>


在 Python 中，元组与列表是非常相似的类型；但主要的区别可能是元组不能在赋值后更改。下述的代码若是针对列表则是可行的，但对于元组则会报错：

In [8]:
tup[1] = plt

TypeError: 'tuple' object does not support item assignment

因此，通常元组可以用作函数的参数或返回值类型。

在便利列表或元组时，有时我们会希望同时得到列表中元素的值与其索引；这时可以使用 Python 内置函数 `enumerate`。该函数是迭代器，在迭代过程中，返回元组。下述例中，`idx, val` 两者就是元组。

In [9]:
for idx, val in enumerate(tup):
    print("index:", idx, "; value: ", val)

index: 0 ; value:  1
index: 1 ; value:  <module 'numpy' from '/mnt/d/Anaconda3WSL/lib/python3.7/site-packages/numpy/__init__.py'>
index: 2 ; value:  range(1, 10, 3)
index: 3 ; value:  Pastafarianism


最后，我们指出，单元素的元组的写法比较特殊。我们可以使用内置函数 `type` 来判断下述的变量是否是元组：

In [10]:
v1 = (1)
v2 = (1,)
print(type(v1), v1)
print(type(v2), v2)

<class 'int'> 1
<class 'tuple'> (1,)


### 字典 (dict)

字典类型的格式与 json 格式非常相像 (但两者的概念完全不同)。尽管字典类似于 C++ STL 的 `std::unordered_map`，但 Python 字典的键或值可以是不同的类型。Python 的字典通过哈希表储存，因此键必须是可哈希的。可哈希的对象不仅是数、字符串，还可以是一般的库、函数、类、元组，但字典、列表等本身不可哈希。

In [11]:
dct = {
    "student id": 1,
    plt: np,
    5: range(1, 10, 3),
    (1, np.ndarray): "Pastafarianism"
}

从字典中索引元素通过方括号运算符 (operator) 就能实现：

In [12]:
dct[(1, np.ndarray)]

'Pastafarianism'

若遍历字典，则可以通过下述代码实现：

In [13]:
for key in dct:
    print("key:", key, "; value:", dct[key])

key: student id ; value: 1
key: <module 'matplotlib.pyplot' from '/mnt/d/Anaconda3WSL/lib/python3.7/site-packages/matplotlib/pyplot.py'> ; value: <module 'numpy' from '/mnt/d/Anaconda3WSL/lib/python3.7/site-packages/numpy/__init__.py'>
key: 5 ; value: range(1, 10, 3)
key: (1, <class 'numpy.ndarray'>) ; value: Pastafarianism


在这份笔记中，字典通常用来储存配置信息。

### 列表的复制

这里列举一些列表复制中会遇到的现象；或者跟一般地，浅复制 (shallow copy) 与深复制 (deep copy) 现象。更原理性的讨论需要参考其它资料。之所以这里会单列一节，是因为 Python 中存在许多隐式的引用；这在其它语言中不存在或者不太常见。

这里使用到 Python 的内置库 [copy](https://docs.python.org/3/library/copy.html)。

我们先考察下述的几个变量：

In [14]:
lst = [[]] * 3
lst1 = lst
lst2 = copy.copy(lst)
lst3 = copy.deepcopy(lst)

其中，`lst1` 与 `lst` 是完全等价的，除非重新定义 `lst1`；而剩下两个变量则略有区别。这从内置函数 `id` 可以看出：

In [15]:
for l in [lst, lst1, lst2, lst3]:
    print(id(l))

139863226765832
139863226765832
139863226768008
139863226764232


即使不是所有列表的 `id` 返回值相同，但确实这些列表是相同的列表：

In [16]:
for l in [lst, lst1, lst2, lst3]:
    print(l)

[[], [], []]
[[], [], []]
[[], [], []]
[[], [], []]


In [17]:
bools = [[None] * 4 for _ in range(4)]  # construct empty list which is free from shallow copy problem
for i1, l1 in enumerate([lst, lst1, lst2, lst3]):
    for i2, l2 in enumerate([lst, lst1, lst2, lst3]):
        bools[i1][i2] = l1 == l2  # output matrix indicates identity between lists
bools

[[True, True, True, True],
 [True, True, True, True],
 [True, True, True, True],
 [True, True, True, True]]

现在我们向 `lst[1]` 的字列表中添加元素：

In [18]:
lst[1].append(1)

我们发现，除了 `lst3` 之外，其它的列表的三个字列表都变成了含有元素的列表。这与 `lst[1].append(1)` 这行程序所表达的意义完全不同：

In [19]:
for l in [lst, lst1, lst2, lst3]:
    print(l)

[[1], [1], [1]]
[[1], [1], [1]]
[[1], [1], [1]]
[[], [], []]


如果我们向 `lst3` 也作类似的操作，我们发现 `lst3` 也产生了相似的效应，但并没有对其它的三个列表也产生影响：

In [20]:
lst3[0].append(2)
for l in [lst, lst1, lst2, lst3]:
    print(l)

[[1], [1], [1]]
[[1], [1], [1]]
[[1], [1], [1]]
[[2], [2], [2]]


这意味着，`copy.deepcopy` 在大多数情况下可以将变量自身的数据与被复制变量的数据区分开，但自身变量之间的引用关系仍然保留。`copy.copy` 有时仍然引用了被复制变量的数据。这也能从调用 `id` 的输出中能看出：

In [21]:
for l in [lst, lst1, lst2, lst3]:
    print([id(val) for val in l])

[139863223829448, 139863223829448, 139863223829448]
[139863223829448, 139863223829448, 139863223829448]
[139863223829448, 139863223829448, 139863223829448]
[139863226766856, 139863226766856, 139863226766856]


同时，统一列表中的三个元素相同，意味着 `[[]] * 3` 事实上是将三个**完全相同**而非**同值**的空列表赋予了列表中。因此，对任何其中一个子空列表添加值将会对其它两个子空列表也产生影响。

下面我们并不是打算添加或更改字列表，而是将列表的元素替换。如果我们执行下述代码：

In [22]:
lst1[0] = 3
lst2[1] = 4
for l in [lst, lst1, lst2]:
    print(l)

[3, [1], [1]]
[3, [1], [1]]
[[1], 4, [1]]


我们注意到，`lst1` 作为 `lst` 的等价变量，两者都会受到元素更变的影响；但 `lst2` 作为 `lst` 的浅复制，则没有受到元素变更的影响。

我们在这里停止对列表复制的讨论。尽管浅复制通常可以减少数据在内存中的复制并提高存储效率，但也会引起一些不直观、不易调试的问题。列表复制这一小节的主要目的是提醒读者，在对包括列表、numpy 向量之内的其它 Python 对象进行操作时，需要小心可能出现的浅复制、引用等问题。同时，我们应当要知道，在 Python 中，没有任何变量是严格可以作为常量传值的；因此即使对于一些元素通过只设定 getting 函数而不设定 setting 函数进行保护，你也很可能会通过浅复制过程将这种被保护的变量的值改写。

<div class="alert alert-info">

**任务**

1. 尽管我们刚才说 `[[]] * 3` 在增添元素 `1` 时会出现期望之外的 `[[1], [1], [1]]`，但我们也使用了看起来非常类似的危险的代码 `bools = [[None] * 4 for _ in range(4)]`；事实上，这段代码在对元素赋值过程中是安全的。请尝试执行下述两个代码块，并解释输出结果。

</div>

In [23]:
bools1 = [[None] * 3 for _ in range(3)]
bools1[1][2] = 0
bools1  # Expected

[[None, None, None], [None, None, 0], [None, None, None]]

In [24]:
bools2 = [[None] * 3] * 3
bools2[1][2] = 0
bools2  # Unexpected

[[None, None, 0], [None, None, 0], [None, None, 0]]